In [1]:
import os
import time
import requests

# Проверяем статус микросервиса
status_url = "http://localhost:8010/ready"
resp = requests.get(status_url)
print(f"Service status: {resp.status_code}")


Service status: 200


In [2]:
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration

# Setup model path.
model_path = "./models/flan-t5-tsa-thor-xl"
# Setup device.
device = "cuda:0"

model = T5ForConditionalGeneration.from_pretrained(model_path, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.to(device)

def ask(prompt):
  inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
  inputs.to(device)
  output = model.generate(**inputs, temperature=1)
  return tokenizer.batch_decode(output, skip_special_tokens=True)[0]

def target_sentiment_extraction(sentence, target):
  # Setup labels.
  labels_list = ['neutral', 'positive', 'negative']
  # Setup Chain-of-Thought
  step1 = f"Given the sentence {sentence}, which specific aspect of {target} is possibly mentioned?"
  aspect = ask(step1)
  step2 = f"{step1}. The mentioned aspect is about {aspect}. Based on the common sense, what is the implicit opinion towards the mentioned aspect of {target}, and why?"
  opinion = ask(step2)
  step3 = f"{step2}. The opinion towards the mentioned aspect of {target} is {opinion}. Based on such opinion, what is the sentiment polarity towards {target}?"
  emotion_state = ask(step3)
  step4 = f"{step3}. The sentiment polarity is {emotion_state}. Based on these contexts, summarize and return the sentiment polarity only, " + "such as: {}.".format(", ".join(labels_list))
  # Return the final response.
  return ask(step4)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import json

# Открываем и читаем JSON-файл
with open("./data/train.json", "r", encoding="utf-8") as file:
    dataset = json.load(file)


{'text': 'Досудебное расследование по факту покупки ЕНПФ пакета облигаций ТОО "Бузгул Аурум" было начато по инициативе Национального банка РК, сообщил директор департамента защиты прав потребителей и финансовых услуг Нацбанка Казахстана Александр Терентьев.\n"Основанием для досудебного расследования стало обращение Национального банка, письмо от 25 ноября 2016 года. Было обращение Национального банка в правоохранительные органы. Нам эта сделка показалась сомнительной, недостаточно корректной, поэтому Нацбанк 25 ноября 2016 года обратился в правоохранительные органы. Это то, что я могу озвучить на сегодня. Идёт следствие, проводится проверка", – сказал Терентьев.\n28 декабря в Нацбанке заявили, что не знают, что стало основанием для проверки ЕНПФ.\n23 декабря факт проведения проверки в АО "Единый накопительный пенсионный фонд" подтвердился. Пресс-служба Национального банка сообщила, что проверку проводят по операциям, совершённым АО "ЕНПФ" в отношении инвестирования собственных активов.

In [ ]:
import pandas as pd
from razdel import sentenize
import requests
import re
from tqdm.auto import tqdm

create_dataset = []
VALID_ENTITY_TYPES = {"PERSON", "ORGANIZATION", "COUNTRY", "PROFESSION", "NATIONALITY"}

#for i in range(141, 150):
for i in tqdm(range(201, 250), desc="Processing documents"):
    if dataset[i]["sentiment"] == "neutral":
        continue

    datatext = dataset[i]
    per_n_text = re.split(r"\n{1,2}", datatext['text'])

    for paragraph in tqdm(per_n_text, desc=f"Doc {i}", leave=False,  total=len(per_n_text)):
        if paragraph.strip():  # Пропустить пустые абзацы
            for sent in sentenize(paragraph):
                if len(sent.text) < 40 or len(sent.text) > 420:
                    continue

                try:
                    response = requests.post("http://localhost:8010/recognize", json=sent.text)
                    if response.status_code != 200:
                        continue

                    item = response.json()
                    text = item["text"]

                    for entity in item.get("ners", []):
                        entity_text = text[entity[0]:entity[1]]
                        entity_type = entity[2]

                        if entity_type in VALID_ENTITY_TYPES:
                            sentence = text
                            target = entity_text

                            try:
                                flant5_response = target_sentiment_extraction(sentence, target)
                            except Exception as e:
                                print(f"⚠️ Ошибка в target_sentiment_extraction: {e}")
                                continue

                            label = 0
                            if flant5_response == "positive":
                                label = 1
                            elif flant5_response == "negative":
                                label = -1

                            create_dataset.append([
                                text, entity_text, entity_type, entity[0], entity[1], label
                            ])
                except Exception as e:
                    print(f"⚠️ Ошибка при запросе или обработке: {e}")


# Создаем DataFrame
df = pd.DataFrame(create_dataset, columns=["sentence", "entity", "entity_tag", "entity_pos_start_rel", "entity_pos_end_rel", "label"])
file_exists = os.path.exists("ner_results_natasha_plus.csv")
# Записываем в CSV, добавляя заголовки только если файла нет
df.to_csv("ner_results_natasha_plus.csv", mode="a", header=not file_exists, index=False, encoding="utf-8", sep="\t")
print("Данные успешно дописаны в файл: ner_results_natasha_plus.csv")



Processing documents:   0%|          | 0/49 [00:00<?, ?it/s]

Doc 201:   0%|          | 0/6 [00:00<?, ?it/s]

Doc 202:   0%|          | 0/10 [00:00<?, ?it/s]

Doc 203:   0%|          | 0/12 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


...

Doc 243:   0%|          | 0/24 [00:00<?, ?it/s]

Doc 245:   0%|          | 0/21 [00:00<?, ?it/s]

Doc 248:   0%|          | 0/12 [00:00<?, ?it/s]

Doc 249:   0%|          | 0/39 [00:00<?, ?it/s]

Данные успешно дописаны в файл: ner_results_natasha_plus.csv
